In [1]:
!git clone https://github.com/WongKinYiu/yolov7.git

^C


In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 7.5/7.5 MB 32.0 MB/s eta 0:00:00
     --------------------------------------- 14.6/14.6 MB 81.8 MB/s eta 0:00:00
     --------------------------------------- 38.1/38.1 MB 72.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 80.7 MB/s eta 0:00:00
     --------------------------------------- 44.0/44.0 MB 92.9 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00
     ---------------------------------------- 525.0/525.0 kB ? eta 0:00:00
     --------------------------------------- 11.1/11.1 MB 81.8 MB/s eta 0:00:00
     ---------------------------------------- 293.3/293.3 kB ? eta 0:00:00
     ---------------------------------------- 470.4/470.4 kB ? eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 131.4 MB/s eta 0:00:00
     ---------------------------------------- 56.1/56.1 kB ? eta 0:00:0


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install -r requirements-gpu.txt

Looking in indexes: https://download.pytorch.org/whl/cu113



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
### Resolve image problems Libpng warning: iCCP: known incorrect sRGB profile.
import os

folder_path = "./data/YoloData"  # Replace with the actual folder path

# Get all files in the folder
files = [file for file in os.listdir(folder_path) if file.endswith(".png")]

# Process each PNG file
for file in files:
    file_path = os.path.join(folder_path, file)
    command = f'magick.exe  {file_path} +profile "ICM" {file_path}'
    os.system(command)

In [2]:
"""#### Split folders ####
# Solo se ejecuta la primera vez

import random
import shutil

def split_files(source_folder, train_folder, val_folder, split_ratio):
    file_list = os.listdir(source_folder)
    random.shuffle(file_list)
    split_index = int(len(file_list) * split_ratio)

    train_files = file_list[:split_index]
    val_files = file_list[split_index:]

    for file_name in train_files:
        if file_name.endswith('.png'):
            image_path = os.path.join(source_folder, file_name)
            label_path = os.path.join(source_folder, file_name.replace('.png', '.txt'))
            shutil.copy(image_path, train_folder)
            shutil.copy(label_path, train_folder)

    for file_name in val_files:
        if file_name.endswith('.png'):
            image_path = os.path.join(source_folder, file_name)
            label_path = os.path.join(source_folder, file_name.replace('.png', '.txt'))
            shutil.copy(image_path, val_folder)
            shutil.copy(label_path, val_folder)

# Specify the paths and split ratio
source_folder = './data/YoloData'
train_folder = './data/train'
val_folder = './data/val'
split_ratio = 0.8  # 80% for training, 20% for validation

# Call the function to split the files
split_files(source_folder, train_folder, val_folder, split_ratio)"""

import random
import shutil
import os

def split_files(source_folder, train_folder, val_folder, test_folder, split_ratio):
    file_list = os.listdir(source_folder)
    random.shuffle(file_list)
    train_split = int(len(file_list) * split_ratio[0])
    val_split = train_split + int(len(file_list) * split_ratio[1])

    train_files = file_list[:train_split]
    val_files = file_list[train_split:val_split]
    test_files = file_list[val_split:]

    for file_name in train_files:
        if file_name.endswith('.png'):
            image_path = os.path.join(source_folder, file_name)
            label_path = os.path.join(source_folder, file_name.replace('.png', '.txt'))
            shutil.copy(image_path, train_folder)
            shutil.copy(label_path, train_folder)

    for file_name in val_files:
        if file_name.endswith('.png'):
            image_path = os.path.join(source_folder, file_name)
            label_path = os.path.join(source_folder, file_name.replace('.png', '.txt'))
            shutil.copy(image_path, val_folder)
            shutil.copy(label_path, val_folder)

    for file_name in test_files:
        if file_name.endswith('.png'):
            image_path = os.path.join(source_folder, file_name)
            label_path = os.path.join(source_folder, file_name.replace('.png', '.txt'))
            shutil.copy(image_path, test_folder)
            shutil.copy(label_path, test_folder)

# Specify the paths and split ratios
source_folder = './data/YoloData'
train_folder = './data/train'
val_folder = './data/val'
test_folder = './data/test'
split_ratio = [0.7, 0.1, 0.2]  # 70% for training, 10% for validation, 20% for testing

# Call the function to split the files
split_files(source_folder, train_folder, val_folder, test_folder, split_ratio)


In [12]:
### Download Yolov7 weights from pretrained coco model
!curl -L -o ./yolov7/yolov7.pt https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  1 72.0M    1  768k    0     0   655k      0  0:01:52  0:00:01  0:01:51  655k
 57 72.0M   57 41.3M    0     0  19.0M      0  0:00:03  0:00:02  0:00:01 40.6M
100 72.0M  100 72.0M    0     0  25.9M      0  0:00:02  0:00:02 --:--:-- 44.5M


In [6]:
!python train.py  --workers 8 --device 0 --batch-size 16 --epochs 300 --img 640 640  --hyp data/hyp.scratch.p5.yaml --name manifestaciones-rupestres --weights yolov7.pt

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 4.55, Best Possible Recall (BPR) = 0.9998
                 all         623        1941      0.0394      0.0548      0.0126     0.00352
                 all         623        1941       0.291       0.224       0.104      0.0331
                 all         623        1941       0.468       0.238       0.149      0.0553
                 all         623        1941       0.495        0.21       0.183      0.0657
                 all         623        1941       0.602       0.257       0.225      0.0856
                 all         623        1941       0.448       0.266       0.186      0.0629
                 all         623        1941       0.468       0.309       0.242       0.101
                 all         623        1941       0.408       0.361       0.237      0.0926
                 all         623        1941       0.445       0.363      

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

   278/299     11.2G   0.02541   0.01179  0.002224   0.03943       146       640:  27%|##6       | 31/116 [00:27<01:13,  1.16it/s]
   278/299     11.2G   0.02541   0.01179  0.002224   0.03943       146       640:  28%|##7       | 32/116 [00:27<01:12,  1.15it/s]
   278/299     11.2G   0.02547   0.01181   0.00219   0.03946       115       640:  28%|##7       | 32/116 [00:28<01:12,  1.15it/s]
   278/299     11.2G   0.02547   0.01181   0.00219   0.03946       115       640:  28%|##8       | 33/116 [00:28<01:12,  1.14it/s]
   278/299     11.2G   0.02542   0.01188  0.002179   0.03948       143       640:  28%|##8       | 33/116 [00:29<01:12,  1.14it/s]
   278/299     11